SQL pairs components parsing for further model training and inference

In [2]:
import sys
sys.path.append('/home/shelunts/thesis/TabularSemantingParsing/')

In [3]:
from parse_sql_one import get_schemas_from_json, Schema
from process_sql import get_sql
from get_tables import convert_fk_index, dump_db_json_schema
import json
import sqlite3
from os.path import isfile, isdir, join, split, exists, splitext
import pandas as pd
import nltk
import re

In [16]:
ROOT_DIR = os.path.dirname(os.path.abspath(''))
DATA_DIR = ROOT_DIR + '/data/spider/database/isrecon/isrecon.sqlite'
SCHEMA_DIR = ROOT_DIR + '/data/spider/tables.json' 
SQL_DIR = ROOT_DIR + '/data/isrecon/isrecon/test_sql_pairs.sql'
DEV_DIR = ROOT_DIR + '/data/spider/dev.json'
DEV_DIR

'/home/shelunts/thesis/TabularSemantingParsing/data/spider/dev.json'

In [5]:
db_id = "isrecon"
schemas, db_names, tables = get_schemas_from_json(SCHEMA_DIR)
schema = schemas[db_id]
table = tables[db_id]
schema = Schema(schema, table)

In [10]:
#opening the file with question-SQL pairs, extracting and parsing different components for json file
with open(SQL_DIR, encoding='utf8') as f:
    l = [line.rstrip('\n') for line in f]
queries = [re.sub(";","",v) for i, v in enumerate(l) if i % 2 != 0] 
l = [v for i, v in enumerate(l) if i % 2 == 0] 
questions = [re.sub('^[^_]*:', '', line).strip() for line in l] #opening the file and selecting only lines that contain SQL queries
question_toks = [nltk.word_tokenize(i) for i in questions] #tokenizing questions according to json format
query_toks = [nltk.word_tokenize(i) for i in queries] #tokenizing queries according to json format
parsed_sqls = [get_sql(schema, i) for i in queries] #parsing sql queries into the format specified in the json file

In [11]:
questions

['What are the papers written by Dorothy Leidner?',
 'How many papers did Dorothy Leidner write?',
 'Who published the most papers in 2020?',
 'How many papers were published by Dorothy Leidner in 2020?',
 'What were the papers published by Dorothy Leidner in 2020?',
 'How many papers were published in 2020?',
 'Show the titles of all the published papers in 2020.',
 'In how many journals did papers of Dorothy Leidner appear?',
 'In which institutions  does Dorothy Leidner work?',
 'In which institutions and departments does Dorothy Leidner work?',
 'What is the number of dsr entities for Dorothy Leidner papers?',
 'Which paper of Dorothy Leidner has the highest number of dsr entities?',
 'List papers published in JIT journal.',
 'How many papers are published in journal with acronym "JIT"?',
 'How many papers were published in JIT journal in 2020?',
 'How many papers were published in JIT journal this year?',
 'How many papers were published in JIT journal last year?',
 'How many pape

In [12]:
#creating keys for json file and respective values from different variables
keys = ('db_id', 'question', 'question_toks',  'query', 'query_toks', 'sql')
db_id = [db_id] * len(questions)
test = list(zip(db_id, questions, question_toks, queries, query_toks, parsed_sqls))

In [13]:
#creating a function that will convert tuples to list later
def listit(t):
    return list(map(listit, t)) if isinstance(t, (list, tuple)) else t

In [14]:
#converting test json values from tuples into list
test_list = []
for i in test:
    lst = listit(i)
    test_list.append(lst)

In [15]:
#creating the file with all json components
list_json = []
for key in range(len(questions)):
        test_json = dict.fromkeys(keys)
        test_json['db_id'] = test_list[key][0]
        test_json['question'] = test_list[key][1]
        test_json['question_toks'] = test_list[key][2]
        test_json['query'] = test_list[key][3]
        test_json['query_toks'] = test_list[key][4]
        test_json['sql'] = test_list[key][5]
        list_json.append(test_json)

In [43]:
#test set should always be named dev.json
with open(DEV_DIR, 'w') as test_file:
    json.dump(list_json, test_file, indent=2, separators=(',', ': '))

In [47]:
# train_spider = json.load(open('/home/shelunts/thesis/TabularSemantingParsing/data/spider/train_spider_full_test4.json'))
# train_combined = list_json + train_spider
# with open('/home/shelunts/thesis/TabularSemantingParsing/data/spider/train_spider&recon.json', 'w') as test_file:
#     json.dump(train_combined, test_file, indent=2, separators=(',', ': '))